In [1]:
import json
from collections import defaultdict
import os
import sys
from textwrap import dedent
import re


In [2]:
with open('05_output.json') as fh:
    data_request_info = json.load(fh)

In [3]:
data_request_info['Compound Name']['Emon.treeFracBdlDcd']

{'cell_measures': 'area: areacella',
 'cell_methods': 'area: time: mean',
 'cmip6_differences': '"cell_methods" was "area: mean where land over all_area_types time: mean"\n',
 'cmip7_compound_name': 'land.mon.treeFracBdlDcd_tavg-u-hxy-u',
 'comment': 'This is the percentage of the entire grid cell that is covered by broadleaf deciduous trees.',
 'dimensions': 'longitude latitude time typetreebd',
 'flags': ['land', 'mon', 'CMIP6'],
 'frequency': 'mon',
 'long_name': 'Broadleaf Deciduous Tree Area Percentage',
 'mapping_info': {'STASH': [{'domain': 'DIAG',
    'stash': 'm01s03i395',
    'stash_code': 3395,
    'time': 'TMONMN',
    'usage': 'UP5'},
   {'domain': 'DTILE',
    'stash': 'm01s19i013',
    'stash_code': 19013,
    'time': 'TMONMN',
    'usage': 'UP5'}],
  'cmip6_status': 'ok',
  'comment': '',
  'consistency': [],
  'expression': 'UKESM1:"land_class_area(m01s19i013[lbtim_ia=240,lbproc=128], m01s03i395[lbproc=128], land_class=\'broadLeafTreeDeciduous\')", HadGEM3:"land_class_

In [4]:
DR_INFO = dedent("""
## Data Request information
   
| Field | Value | Notes |
| --- | --- | --- |
| Key | {key} | |
| Branded variable name | {branded_variable_name} | |
| CF standard name | {standard_name} | |
| Long name | {long_name} | |
| Comment | {comment} | |
| Dimensions | {dimensions} | |
| Frequency | {frequency} | |
| Units | {units} | |
| Positive | {positive} | |
| Processing notes | {processing_note} | |
| Cell methods | {cell_methods} | |
| Cell measures | {cell_measures} | |
| Modeling realm | {modeling_realm} | |
| Variable status | {variable_status} | |
| CMIP6 Differences | {cmip6_differences} | |

""")

MAPPING = dedent("""
## Mapping information 

| Field | Value | Notes |
| --- | --- | --- |
""")

STASH = dedent("""
## STASH entries (relevant for UM only)

| Model | STASH | Section, item number | Time Profile | Domain Profile | Usage Profile |
| --- | --- | --- | --- | --- | --- |
{stash_table}
""")

In [5]:
commands = []
label_set = set()
for key, entry in data_request_info['Compound Name'].items():

    entry['branded_variable_name'] = entry['cmip7_compound_name'].split('.')[-1]
    labels = [entry['modeling_realm'], entry['frequency']]
    if 'CMIP6 CMOR' in entry['variable_status']:
        labels.append('CMIP6')
        
    title = "Variable {cmip7_compound_name} ({key})".format(key=key,**entry)
    entry['processing_note'] = entry['processing_note'].replace("\n", "; ")
    entry['cmip6_differences'] = entry['cmip6_differences'].replace("\n", "; ")
    body = DR_INFO.format(key=key, **entry)

    expression = entry['mapping_info'].get('expression', '')
    model = entry['mapping_info'].get('model', '')
    model_units = entry['mapping_info'].get('model_units', '')

    if model == 'PER_MODEL' or model == 'BOTH':
        labels += ['UKESM1', 'HadGEM3-GC31']
    elif model == '':
        pass
    else:
        labels.append(model)

    if 'BaselineClimateVariable' in entry['flags']:
        labels.append('BCV')
    
        
    for l in labels:
        label_set.add(l)
    
    #body += MAPPING.format(expression=expression, model=model, units=model_units)

    mapping = MAPPING
    if key == "Emon.treeFracNdlEvg":
        print(model)
    if model == 'BOTH' or model == 'UKESM1':
        if key == "Emon.treeFracNdlEvg":
            print(1)
        mapping += "| Expression UKESM1 | `{}` | |\n".format(expression)
    if model == 'BOTH' or model == 'HadGEM3-GC31':
        mapping += "| Expression HadGEM3-GC31 | `{}` | |\n".format(expression)
    if model == 'PER_MODEL':
        match = re.search('UKESM1:"(.*)", HadGEM3:"(.*)"', expression)
        if key == "Emon.treeFracNdlEvg":
            print(expression)
            print(match)
        
        mapping += "| Expression UKESM1 | `{}` | |\n".format(match.group(1))
        mapping += "| Expression HadGEM3-GC31 | `{}` | |\n".format(match.group(1))

        
    mapping += "| Model units | {} | |\n".format(model_units)
    mapping += "\n"
        
    body += mapping   
        
    stash_table = ''

    if 'STASH' in entry['mapping_info']:
        if model in ['UKESM1', 'BOTH', 'PER_MODEL']:
            stash_table += '\n'.join([
                "| UKESM1 | {stash} | {stash_code:,} | {time} | {domain} | {usage} |".format(**i) for i in entry['mapping_info']['STASH']])
            stash_table += "\n"
        if model in ['HadGEM3', 'BOTH', 'PER_MODEL']:
            stash_table += '\n'.join([
                "| HadGEM3-GC31 | {stash} | {stash_code:,} | {time} | {domain} | {usage} |".format(**i) for i in entry['mapping_info']['STASH']]) 
            stash_table += "\n"
    if stash_table == "":
            stash_table = "| | | | | |"
    body += STASH.format(stash_table=stash_table,**entry['mapping_info'])

    bodyfile = 'issues/{}'.format(key) 
    with open(bodyfile, 'w') as fh:
        fh.write(dedent(body))
    
    commands.append('gh issue create --title "{title}" --body-file {bodyfile} --label {labels}'.format(
        title=title, bodyfile=bodyfile, labels=" --label ".join(labels)))

PER_MODEL
UKESM1:"land_class_area(m01s19i013[lbtim_ia=240,lbproc=128], m01s03i395[lbproc=128], land_class='needleLeafTreeEvergreen')", HadGEM3:"land_class_area(m01s03i317[lbproc=128], m01s03i395[lbproc=128], land_class='needleLeafTreeEvergreen')"
<re.Match object; span=(0, 236), match='UKESM1:"land_class_area(m01s19i013[lbtim_ia=240,l>


In [6]:
for i in label_set:
    print('gh label create {} -d ""'.format(i))

gh label create 1hr -d ""
gh label create BCV -d ""
gh label create fx -d ""
gh label create atmos -d ""
gh label create 3hr -d ""
gh label create aerosol -d ""
gh label create mon -d ""
gh label create yr -d ""
gh label create subhr -d ""
gh label create land -d ""
gh label create ocnBgchem -d ""
gh label create atmosChem -d ""
gh label create seaIce -d ""
gh label create UKESM1 -d ""
gh label create landIce -d ""
gh label create dec -d ""
gh label create day -d ""
gh label create CMIP6 -d ""
gh label create ocean -d ""
gh label create 6hr -d ""
gh label create HadGEM3-GC31 -d ""


In [7]:
with open('commands.txt', 'w') as fh:
    for c in commands:
        fh.write(c + "\n")

In [8]:
!grep Amon commands.txt | grep BCV

gh issue create --title "Variable atmos.mon.cl_tavg-l-hxy-u (Amon.cl)" --body-file issues/Amon.cl --label atmos --label mon --label CMIP6 --label UKESM1 --label HadGEM3-GC31 --label BCV
gh issue create --title "Variable atmos.mon.cli_tavg-l-hxy-u (Amon.cli)" --body-file issues/Amon.cli --label atmos --label mon --label CMIP6 --label UKESM1 --label HadGEM3-GC31 --label BCV
gh issue create --title "Variable atmos.mon.clivi_tavg-u-hxy-u (Amon.clivi)" --body-file issues/Amon.clivi --label atmos --label mon --label CMIP6 --label UKESM1 --label HadGEM3-GC31 --label BCV
gh issue create --title "Variable atmos.mon.clt_tavg-u-hxy-u (Amon.clt)" --body-file issues/Amon.clt --label atmos --label mon --label CMIP6 --label UKESM1 --label HadGEM3-GC31 --label BCV
gh issue create --title "Variable atmos.mon.clw_tavg-l-hxy-u (Amon.clw)" --body-file issues/Amon.clw --label atmos --label mon --label CMIP6 --label UKESM1 --label HadGEM3-GC31 --label BCV
gh issue create --title "Variable atmos.mon.clwvi_ta